# Testing the models

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

## Load the test data

In [3]:
def process_image_gender(path_gender, label):
    # Desired size
    size = 250
    path, gender = path_gender
    # Get the image
    img = tf.io.read_file(path)
    # Decode the PNG
    img = tf.image.decode_png(img)
    # Resize image
    img = tf.image.resize(img, (size, size))
    # Reshape image (this is not necessary but I do it so that I don't need to be modifying the shape in the input layer)
    img = tf.reshape(img, [size, size, 1])
    # Cast image to float32
    img = tf.cast(img, tf.float32)
    # Normalize image
    img = img/255.0

    return (img, gender), label
def process_image(path, label):
    # Desired size
    size = 250
    # Get the image
    img = tf.io.read_file(path)
    # Decode the PNG
    img = tf.image.decode_png(img)
    # Resize image
    img = tf.image.resize(img, (size, size))
    # Reshape image (this is not necessary but I do it so that I don't need to be modifying the shape in the input layer)
    img = tf.reshape(img, [size, size, 1])
    # Cast image to float32
    img = tf.cast(img, tf.float32)
    # Normalize image
    img = img/255.0

    return img, label
def get_paths_n_labels(csv_path, images_folder_path, id_col, label_col):
    df = pd.read_csv(csv_path)
    paths = [images_folder_path + '/' + str(id) + '.png' for id in df[id_col].tolist()]
    labels = df[label_col].tolist()
    return paths, labels
def get_data(csv_path, images_folder_path, id_col, label_col, gender_col):
    df = pd.read_csv(csv_path)
    paths = [images_folder_path + '/' + str(id) + '.png' for id in df[id_col].tolist()]
    labels = df[label_col].tolist()
    gender = [0 if (str(g)=="False" or str(g)=="FALSE" or str(g)=="F") else 1 for g in df[gender_col].tolist()] # male 1, female 0
    return paths, labels, gender

In [4]:
# original_r250p
otest_paths, otest_labels, otest_gender = get_data(
                                                csv_path='./data/pre_processed/test/test.csv',
                                                images_folder_path='./data/pre_processed/test/{}'.format('original_r250p'),
                                                id_col='Case ID',
                                                label_col='Ground truth bone age (months)',
                                                gender_col='Sex')
# Without gender
test_dataset_orig = tf.data.Dataset.from_tensor_slices((otest_paths,otest_labels)).map(process_image).batch(32)
# With gender
test_dataset_orig_gender = tf.data.Dataset.from_tensor_slices(((otest_paths,otest_gender),otest_labels)).map(process_image_gender).batch(32)

In [5]:
# preprocessed_r250p
ptest_paths, ptest_labels, ptest_gender = get_data(
                                                csv_path='./data/pre_processed/test/test.csv',
                                                images_folder_path='./data/pre_processed/test/{}'.format('preprocessed_r250p'),
                                                id_col='Case ID',
                                                label_col='Ground truth bone age (months)',
                                                gender_col='Sex')
# Without gender
test_dataset_preproc = tf.data.Dataset.from_tensor_slices((ptest_paths,ptest_labels)).map(process_image).batch(32)
# With gender
test_dataset_preproc_gender = tf.data.Dataset.from_tensor_slices(((ptest_paths,ptest_gender),ptest_labels)).map(process_image_gender).batch(32)

## Load the models

In [6]:
# Load the models
inceptionv4_gender_orig = tf.keras.models.load_model('./models/inceptionv4_gender__original_r250p_1676203916.h5')
inceptionv4_gender_preproc = tf.keras.models.load_model('./models/inceptionv4_gender__preprocessed_r250p_1676244085.h5')
vgg6_gender_orig = tf.keras.models.load_model('./models/vgg_gender_original_r250p_1676150023.h5')
vgg6_gender_preproc = tf.keras.models.load_model('./models/vgg_gender_preprocessed_r250p_1676154512.h5')

## Evaluate the models

In [7]:
# Evaluate models
def evaluate_model_gender(model, dataset):
    p_test = model.predict(dataset).reshape(-1)
    y_test = np.array([label.numpy() for (img,gender), label in dataset.unbatch()])
    mae = np.mean(np.abs(np.subtract(y_test, p_test)))
    print("Mean absolute error: "+str(mae))
def evaluate_model(model, dataset):
    p_test = model.predict(dataset).reshape(-1)
    y_test = np.array([label.numpy() for img, label in dataset.unbatch()])
    mae = np.mean(np.abs(np.subtract(y_test, p_test)))
    print("Mean absolute error: "+str(mae))

### VGG-6

w/o Gender


In [8]:
# original_r250p

In [9]:
# preprocessed_r250p

w/ Gender

In [10]:
# original_r250p
evaluate_model_gender(vgg6_gender_orig,test_dataset_orig_gender)

7/7 [==============================] - 3s 44ms/step
Mean absolute error: 15.715706


In [11]:
# preprocessed_r250p
evaluate_model_gender(vgg6_gender_preproc,test_dataset_preproc_gender)

7/7 [==============================] - 0s 18ms/step
Mean absolute error: 9.180346


### Inception-V4

w/o Gender


In [12]:
# original_r250p

In [13]:
# preprocessed_r250p

w/ Gender

In [14]:
# original_r250p
evaluate_model_gender(inceptionv4_gender_orig,test_dataset_orig_gender)

7/7 [==============================] - 4s 260ms/step
Mean absolute error: 8.0534935


In [15]:
# preprocessed_r250p
evaluate_model_gender(inceptionv4_gender_preproc,test_dataset_preproc_gender)

7/7 [==============================] - 3s 275ms/step
Mean absolute error: 7.495013
